<h1>Parsing OMA website</h1>

<p>This notebook is just a prototype for testing pieces of python code aimed to parse a specific website.</p>

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup
import pandas as pd
import re

In [2]:
url = 'https://www.oma.by/catalog/'
#opens up a connection, grabs the page and downloads it
oma_main_html = requests.get(url)

<p>soup is a BeautifulSoup object which represents the document as a nested data sctructure:</p>

In [3]:
oma_catalog_soup = BSoup(oma_main_html.text, 'html.parser')
oma_catalog_soup.title
oma_catalog_soup.close

<h3>Getting categories </h3>
<p>So we get the soup object successfully. Now will try to find category headers on a catalog page (there are 7 main categories). </p>
<img src="screenshots/category_schema.png" align="left" width="600">

In [4]:
categories = oma_catalog_soup.select('section.bordered-section h2')
print(f'There are {len(categories)} objects that has h2 tags inside the header tag')

print('\nThe whole list of the categories:')
for category in categories:
    print(category)

There are 7 objects that has h2 tags inside the header tag

The whole list of the categories:
<h2>Инструменты, крепёж</h2>
<h2>Отделка</h2>
<h2>Садовый центр / отдых</h2>
<h2>Сантехника</h2>
<h2>Строительство</h2>
<h2>Товары для дома</h2>
<h2>Электротехника</h2>


<h3>Getting all categories</h3>
<p>So we get the categories list correctly and we can use this h2 tags as anchors for looping.</p>
<img src="screenshots/subcategories_schema.png" align="left" width="720">

<p>Getting subcategories lvl1 and subcategores lvl2 is possible by processing each category one by one. It is useful to extract 7 category sections for that. </p>

In [5]:
categories = oma_catalog_soup.findAll('section',\
                                      {'class':'bordered-section js-accordion-group'})

print(f'\nThe type of categories is {type(categories)}.')

print(f'\nThere are {len(categories)} main category objects in {type(categories)}.')

print(f'\nEach category object has {type(categories[0])} type.')

#strip the name of the category
category_name_0 = categories[0].select('section.bordered-section h2')

print('\nFirst category raw:')
print(category_name_0)
print(type(category_name_0))

print('\nFirst category extracted from list:')
print(category_name_0[0])
print(type(category_name_0[0]))

print('\nFirst category extracted from list and converted to string:')
print(str(category_name_0[0]))
print(type(str(category_name_0[0])))
print('')


The type of categories is <class 'bs4.element.ResultSet'>.

There are 7 main category objects in <class 'bs4.element.ResultSet'>.

Each category object has <class 'bs4.element.Tag'> type.

First category raw:
[<h2>Инструменты, крепёж</h2>]
<class 'list'>

First category extracted from list:
<h2>Инструменты, крепёж</h2>
<class 'bs4.element.Tag'>

First category extracted from list and converted to string:
<h2>Инструменты, крепёж</h2>
<class 'str'>



<p>Extract category names:</p>

In [6]:
def remove_tags(text): 
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def extract_category_name(list):
    name_with_tags = str(list[0])
    return remove_tags(name_with_tags)

for category in categories:
    category_name_raw = category.select('section.bordered-section h2')
    print(extract_category_name(category_name_raw))

Инструменты, крепёж
Отделка
Садовый центр / отдых
Сантехника
Строительство
Товары для дома
Электротехника


<h3>Getting subcategories lvl_1</h3>
<p>Getting subcategories lvl_1 fron single category object.</p>

In [7]:
subcats_lvl_1 = categories[0].findAll('div',\
                                      {'class':'catalog-all-item'})

print(f'\nThe type of subcats_lvl_1 is {type(subcats_lvl_1)}.')

print(f'\nThere are {len(subcats_lvl_1)} subcats_lvl_1 objects')

print(f'\nEach subcats_lvl_1 object has {type(subcats_lvl_1[0])} type.')

print('\n\n first subcategory_lvl_1:')
print(subcats_lvl_1[0])

print(subcats_lvl_1[0].select('div.accordion-item_title a'))


The type of subcats_lvl_1 is <class 'bs4.element.ResultSet'>.

There are 40 subcats_lvl_1 objects

Each subcats_lvl_1 object has <class 'bs4.element.Tag'> type.


 first subcategory_lvl_1:
<div class="catalog-all-item col col-lg-3-of-12 col-md-4-of-12 col-sm-6-of-12 col-xs-12-of-12">
<div class="catalog-all-item_img-box">
<img alt="" src="/upload/Sh/imageCache/ad0/5e0/14b9b1eb45799ec953c4d4a00d601b13.jpg"/>
</div>
<div class="accordion-item accordion-item__right-icon js-accordion">
<div class="accordion-item_top">
<div class="accordion-item_icon css-plus-icon js-accordion-bar"></div>
<div class="accordion-item_title">
<a href="/ankery-13127-c">Анкеры</a>
</div>
</div>
<div class="accordion-item_body js-accordion-body js-show-more-box" data-items-visible="11">
<a class="section-submenu-sublink" href="/ankery-ramnye-13251-c">Анкеры рамные</a>
<a class="section-submenu-sublink" href="/ankery-spetsialnye-13252-c">Анкеры специальные</a>
<a class="section-submenu-sublink" href="/bolty-anker

<p>Extract subcats_lvl_1 names from first <i>category</i> object:</p>

In [8]:
subcats_lvl_1_divs = categories[0].findAll('div',\
                                      {'class':'accordion-item_title'})

print(f'\nThere are {len(subcats_lvl_1_divs)} subcategory lvl_1 divs')

subcats_lvl_1_a_tags = categories[0].select('div.accordion-item_title a')
print(f'\nThere are {len(subcats_lvl_1_a_tags)} subcategory lvl_1 a tags')

print(f'\nThere first subcategory lvl_1 tag: {subcats_lvl_1_a_tags[0]}')

print('\n\nFirst 3 subcategory lvl_1 tags:')
for subcat_lvl_1_a_tag in subcats_lvl_1_a_tags[:3]:
    print(f'\n{subcat_lvl_1_a_tag}')
    
print('\n\nThe last 4 subcategory lvl_1 tag names:')
for subcat_lvl_1_a_tag in subcats_lvl_1_a_tags[-4:]:
    print(f'\n{remove_tags(str(subcat_lvl_1_a_tag))}')


There are 40 subcategory lvl_1 divs

There are 40 subcategory lvl_1 a tags

There first subcategory lvl_1 tag: <a href="/ankery-13127-c">Анкеры</a>


First 3 subcategory lvl_1 tags:

<a href="/ankery-13127-c">Анкеры</a>

<a href="/benzokosy-travokosilki-elektrotrimmery-14309-c">Бензокосы(травокосилки)/электротриммеры</a>

<a href="/bolty-vinty-gayki-shayby-shpilki-konfirmaty-13128-c">Болты/ винты/ гайки/ шайбы/шпильки/конфирматы</a>


The last 4 subcategory lvl_1 tag names:

Шлифмашины, полирователи, бороздоделы

Шурупы/ заглушки

Электросварочное оборудование и материалы

Ящики для инструмента , органайзеры


<p>The last 4 elements of subcategories 1lvl.</p>
<img src="screenshots/subcats_lvl_1.png" align="left" width="720">

<h3>Getting subcategories lvl_2</h3>
<p>Getting subcategories lvl_2 from single subcats_lvl_1 object.</p>

In [9]:
subcats_lvl_2_tags = subcats_lvl_1[1].findAll('a',\
                                      {'class':'section-submenu-sublink'})

print(f'\nThe type of subcats_lvl_2 is {type(subcats_lvl_2_tags)}.')

print(f'\n\nThere are {len(subcats_lvl_2_tags)} subcats_lvl_2 objects')

print(f'\n\nEach subcats_lvl_2 object has {type(subcats_lvl_2_tags[0])} type.')

print('\n\nSubcatergoy lvl_2 objects')
for subcat_lvl_2_tag in subcats_lvl_2_tags[:3]:
    print(f'{subcat_lvl_2_tag}')

print('\n\nSubcategory lvl_2 tag names:')
for subcat_lvl_2_a_tag in subcats_lvl_2_tags[-3:]:
    print(f'\n{remove_tags(str(subcat_lvl_2_a_tag))}')
    
print('\n\nSubcategory lvl_2 tag links:')
for subcat_lvl_2_a_tag in subcats_lvl_2_tags[-3:]:
    link = subcat_lvl_2_a_tag.get('href')
    print(f'\n {link}')    

    
#soup.findAll('a', attrs={'href': re.compile("^http://")})


The type of subcats_lvl_2 is <class 'bs4.element.ResultSet'>.


There are 5 subcats_lvl_2 objects


Each subcats_lvl_2 object has <class 'bs4.element.Tag'> type.


Subcatergoy lvl_2 objects
<a class="section-submenu-sublink" href="/benzokosy-travokosilki-bytovye-14324-c">Бензокосы (травокосилки) бытовые</a>
<a class="section-submenu-sublink" href="/benzokosy-travokosilki-professionalnye-14325-c">Бензокосы (травокосилки) профессиональные</a>
<a class="section-submenu-sublink" href="/trimmery-akkumulyatornye-14326-c">Триммеры аккумуляторные</a>


Subcategory lvl_2 tag names:

Триммеры аккумуляторные

Триммеры электрические с верхним двигателем

Триммеры электрические с нижним двигателем


Subcategory lvl_2 tag links:

 /trimmery-akkumulyatornye-14326-c

 /trimmery-elektricheskie-s-verkhnim-dvigatelem-14327-c

 /trimmery-elektricheskie-s-nizhnim-dvigatelem-14328-c


<h3>Create a dataframe for links storage</h3>
<p>Seems like a good idea to have dataframe containing all links to subcategories lvl 2. The dataframe is expected to be lightweight and unseful for future parsing.</p>

In [10]:
main_page_links_df = pd.DataFrame(columns = ['Link number', 'Category',\
                                             'Subcategory lvl 1','Subcategory lvl 2',\
                                             'Link']) 
main_page_links_df

,Link number,Category,Subcategory lvl 1,Subcategory lvl 2,Link


<h3>Create a loop for iterating over categories and subcategories</h3>
<p>Seems like a good idea to have dataframe containing all links to subcategories lvl 2. The dataframe is expected to be lightweight and unseful for future parsing.</p>

In [12]:
%%time
i = 0;
for category in categories:
    category_name_raw = category.select('section.bordered-section h2')
    category_name = extract_category_name(category_name_raw)
    subcats_lvl_1 = category.findAll('div',\
                                      {'class':'catalog-all-item'})
    for subcat_lvl_1 in subcats_lvl_1:
        subcat_lvl_1_name_raw = subcat_lvl_1.select('div.accordion-item_title a')
        subcat_lvl_1_name = extract_category_name(subcat_lvl_1_name_raw)
        subcats_lvl_2_tags = subcat_lvl_1.findAll('a',\
                                      {'class':'section-submenu-sublink'})
        
        for subcat_lvl_2_tag in subcats_lvl_2_tags:
            subcat_lvl_2_name = remove_tags(str(subcat_lvl_2_tag))
            link = 'https://www.oma.by' + subcat_lvl_2_tag.get('href')
            print(i, category_name,' ',subcat_lvl_1_name,' '\
                  , subcat_lvl_2_name,' ',link)
            i += 1;


0 Инструменты, крепёж   Анкеры   Анкеры рамные   https://www.oma.by/ankery-ramnye-13251-c
1 Инструменты, крепёж   Анкеры   Анкеры специальные   https://www.oma.by/ankery-spetsialnye-13252-c
2 Инструменты, крепёж   Анкеры   Болты анкерные   https://www.oma.by/bolty-ankernye-13253-c
3 Инструменты, крепёж   Бензокосы(травокосилки)/электротриммеры   Бензокосы (травокосилки) бытовые   https://www.oma.by/benzokosy-travokosilki-bytovye-14324-c
4 Инструменты, крепёж   Бензокосы(травокосилки)/электротриммеры   Бензокосы (травокосилки) профессиональные   https://www.oma.by/benzokosy-travokosilki-professionalnye-14325-c
5 Инструменты, крепёж   Бензокосы(травокосилки)/электротриммеры   Триммеры аккумуляторные   https://www.oma.by/trimmery-akkumulyatornye-14326-c
6 Инструменты, крепёж   Бензокосы(травокосилки)/электротриммеры   Триммеры электрические с верхним двигателем   https://www.oma.by/trimmery-elektricheskie-s-verkhnim-dvigatelem-14327-c
7 Инструменты, крепёж   Бензокосы(травокосилки)/электр

141 Инструменты, крепёж   Оснастка для зачистки, шлифовки и полировки   Шлифштифты (шарошки)   https://www.oma.by/shlifshtifty-sharoshki-14042-c
142 Инструменты, крепёж   Оснастка для зачистки, шлифовки и полировки   Щетки проволочные и нейлоновые   https://www.oma.by/shchetki-provolochnye-i-neylonovye-14043-c
143 Инструменты, крепёж   Оснастка для пиления, строгания, фрезерования   Ножи к электрорубанкам   https://www.oma.by/nozhi-k-elektrorubankam-13385-c
144 Инструменты, крепёж   Оснастка для пиления, строгания, фрезерования   Ножи фуговальные к станкам   https://www.oma.by/nozhi-fugovalnye-k-stankam-13386-c
145 Инструменты, крепёж   Оснастка для пиления, строгания, фрезерования   Пилки к лобзикам / ножовкам   https://www.oma.by/pilki-k-lobzikam-nozhovkam-13387-c
146 Инструменты, крепёж   Оснастка для пиления, строгания, фрезерования   Пилы дисковые (посадка 32мм и выше) к станкам   https://www.oma.by/pily-diskovye-posadka-32mm-i-vyshe-k-stankam-13388-c
147 Инструменты, крепёж   Осн

250 Инструменты, крепёж   Станки и оборудование   Станки точильно-шлифовальные   https://www.oma.by/stanki-tochilno-shlifovalnye-13343-c
251 Инструменты, крепёж   Столярный инструмент   Ножовки (по дереву, гипсокартону, газосиликату), пилы 2-ручные, лучковые   https://www.oma.by/nozhovki-po-derevu-gipsokartonu-gazosilikatu-pily-2-ruchnye-luchkovye-13476-c
252 Инструменты, крепёж   Столярный инструмент   Рубанки/ фуганки/ фальцгебели/ зензубели и т.п.   https://www.oma.by/rubanki-fuganki-faltsgebeli-zenzubeli-i-t-p-13478-c
253 Инструменты, крепёж   Столярный инструмент   Стамески, долота, резцы   https://www.oma.by/stameski-dolota-reztsy-13479-c
254 Инструменты, крепёж   Столярный инструмент   Топоры, топорища   https://www.oma.by/topory-toporishcha-13480-c
255 Инструменты, крепёж   Строительный инструмент   Гвоздодеры, ломы, зубила   https://www.oma.by/gvozdodery-lomy-zubila-13806-c
256 Инструменты, крепёж   Строительный инструмент   Кельмы   https://www.oma.by/kelmy-13807-c
257 Инстру

349 Отделка   Краски для наружных работ   Краски фасадные   https://www.oma.by/kraski-fasadnye-16166-c
350 Отделка   Лаки   Лак яхтовый   https://www.oma.by/lak-yakhtovyy-13909-c
351 Отделка   Лаки   Лаки для металла и бетона   https://www.oma.by/laki-dlya-metalla-betona-13910-c
352 Отделка   Лаки   Лаки для пола и паркета   https://www.oma.by/laki-dlya-pola-i-parketa-13911-c
353 Отделка   Лаки   Лаки универсальные, мебельные    https://www.oma.by/laki-universalnye-mebelnye-13912-c
354 Отделка   Ламинированный пол   31 класс   https://www.oma.by/31-klass-14015-c
355 Отделка   Ламинированный пол   32 класс   https://www.oma.by/32-klass-14016-c
356 Отделка   Ламинированный пол   33 класс   https://www.oma.by/33-klass-14017-c
357 Отделка   Ламинированный пол   Воск мебельный   https://www.oma.by/vosk-mebelnyy-14018-c
358 Отделка   Ламинированный пол   Средства для укладки ламината   https://www.oma.by/sredstva-dlya-ukladki-laminata-14020-c
359 Отделка   Линолеум   Гомогенное покрытие   ht

520 Садовый центр / отдых   Садовая химия   Защита от насекомых   https://www.oma.by/zashchita-ot-nasekomykh-13873-c
521 Садовый центр / отдых   Садовая химия   Защита от сорняков   https://www.oma.by/zashchita-ot-sornyakov-13874-c
522 Садовый центр / отдых   Садовая химия   Побелка   https://www.oma.by/pobelka-13875-c
523 Садовый центр / отдых   Садовый декор   Арки металлические   https://www.oma.by/arki-metallicheskie-13880-c
524 Садовый центр / отдых   Садовый декор   Брезент   https://www.oma.by/brezent-13881-c
525 Садовый центр / отдых   Садовый декор   Заборчики садовые   https://www.oma.by/zaborchiki-sadovye-13882-c
526 Садовый центр / отдых   Садовый декор   Ленты для края газона   https://www.oma.by/lenty-dlya-kraya-gazona-13883-c
527 Садовый центр / отдых   Садовый декор   Плитка /бордюр/ступень   https://www.oma.by/plitka-bordyur-stupen-13884-c
528 Садовый центр / отдых   Садовый декор   Пруды, фонтаны   https://www.oma.by/prudy-fontany-13886-c
529 Садовый центр / отдых   С

...

756 Строительство   Кровельные материалы   Металлочерепица и доборные элементы   https://www.oma.by/metallocherepitsa-i-dobornye-elementy-13349-c
757 Строительство   Кровельные материалы   Наплавляемая кровля   https://www.oma.by/naplavlyaemaya-krovlya-13350-c
758 Строительство   Кровельные материалы   Пленки и мембраны кровельные   https://www.oma.by/plenki-i-membrany-krovelnye-13348-c
759 Строительство   Кровельные материалы   Поликарбонат и доборные элементы   https://www.oma.by/polikarbonat-i-dobornye-elementy-15796-c
760 Строительство   Кровельные материалы   Рубероид, пергамин   https://www.oma.by/ruberoid-pergamin-13351-c
761 Строительство   Кровельные материалы   Холодные мастики, праймер   https://www.oma.by/kholodnye-mastiki-praymer-13352-c
762 Строительство   Кровельные материалы   Шифер   https://www.oma.by/shifer-16395-c
763 Строительство   Лестницы   Лестницы винтовые   https://www.oma.by/lestnitsy-vintovye-13779-c
764 Строительство   Лестницы   Лестницы чердачные   https

844 Строительство   Спецодежда   Брюки, полукомбинезоны утепленные   https://www.oma.by/bryuki-polukombinezony-uteplennye-13596-c
845 Строительство   Спецодежда   Головные уборы   https://www.oma.by/golovnye-ubory-15567-c
846 Строительство   Спецодежда   Куртки утепленные   https://www.oma.by/kurtki-uteplennye-13597-c
847 Строительство   Спецодежда   Полукомбинезоны, брюки   https://www.oma.by/polukombinezony-bryuki-13598-c
848 Строительство   Спецодежда   Рабочие костюмы   https://www.oma.by/rabochie-kostyumy-13599-c
849 Строительство   Спецодежда   Рабочие куртки   https://www.oma.by/rabochie-kurtki-13600-c
850 Строительство   Спецодежда   Рабочие халаты мужские   https://www.oma.by/rabochie-khalaty-muzhskie-13602-c
851 Строительство   Спецодежда   Спецодежда для сварщиков   https://www.oma.by/spetsodezhda-dlya-svarshchikov-13603-c
852 Строительство   Средства защиты   Наушники, каски   https://www.oma.by/naushniki-kaski-14295-c
853 Строительство   Средства защиты   Очки защитные, щи

932 Товары для дома   Кухонная бытовая техника   Печи микроволновые   https://www.oma.by/mikrovolnovki-13954-c
933 Товары для дома   Кухонная бытовая техника   Плитки электрические   https://www.oma.by/plitki-elektricheskie-14911-c
934 Товары для дома   Кухонная бытовая техника   Соковыжималки   https://www.oma.by/sokovyzhimalki-13958-c
935 Товары для дома   Кухонная бытовая техника   Сушилки для овощей   https://www.oma.by/sushilki-dlya-ovoshchey-14912-c
936 Товары для дома   Кухонная бытовая техника   Электрочайники   https://www.oma.by/elektrochayniki-13960-c
937 Товары для дома   Мебель  для ванных комнат   Шкафы, пеналы   https://www.oma.by/shkafy-penaly-14117-c
938 Товары для дома   Мебель для прихожих   Банкетки   https://www.oma.by/banketki-15981-c
939 Товары для дома   Мебель для прихожих   Вешалки напольные   https://www.oma.by/veshalki-napolnye-15982-c
940 Товары для дома   Мебель для прихожих   Тумбы, комоды   https://www.oma.by/tumby-komody-15983-c
941 Товары для дома   Ме

1047 Товары для дома   Товары для уборки   Ветошь   https://www.oma.by/vetosh-14876-c
1048 Товары для дома   Товары для уборки   Губки для мытья   https://www.oma.by/myte-posudy-13471-c
1049 Товары для дома   Товары для уборки   Ерши для унитаза/вантузы   https://www.oma.by/prinadlezhnosti-dlya-sanuzlov-13473-c
1050 Товары для дома   Товары для уборки   Комплекты для мытья полов   https://www.oma.by/komplekty-dlya-mytya-polov-14877-c
1051 Товары для дома   Товары для уборки   Метлы уличные   https://www.oma.by/metly-ulichnye-15938-c
1052 Товары для дома   Товары для уборки   Мешки для мусора   https://www.oma.by/meshki-d-musora-13470-c
1053 Товары для дома   Товары для уборки   Салфетки для уборки   https://www.oma.by/salfetki-dlya-uborki-14879-c
1054 Товары для дома   Товары для уборки   Сменные насадки/рукоятки   https://www.oma.by/smennye-nasadki-rukoyatki-14881-c
1055 Товары для дома   Товары для уборки   Стеклоочистители/ пылевыбивалки   https://www.oma.by/stekloochistiteli-pylevy

1119 Электротехника   Розетки и выключатели   Bironi (откр)   https://www.oma.by/bironi-otkr-16507-c
1120 Электротехника   Розетки и выключатели   Bylectrica гармония (скр)   https://www.oma.by/bylectrica-garmoniya-skr-16508-c
1121 Электротехника   Розетки и выключатели   Bylectrica гармония люкс (скр)   https://www.oma.by/bylectrica-garmoniya-lyuks-skr-16509-c
1122 Электротехника   Розетки и выключатели   Bylectrica открытой установки   https://www.oma.by/bylectrica-otkrytoy-ustanovki-16510-c
1123 Электротехника   Розетки и выключатели   Bylectrica стиль (скр)   https://www.oma.by/bylectrica-stil-skr-16511-c
1124 Электротехника   Розетки и выключатели   Bylectrica уют (скр)   https://www.oma.by/bylectrica-uyut-skr-16512-c
1125 Электротехника   Розетки и выключатели   El-bi zena (скр)   https://www.oma.by/el-bi-zena-skr-16513-c
1126 Электротехника   Розетки и выключатели   El-bi zirve (скр)   https://www.oma.by/el-bi-zirve-skr-16514-c
1127 Электротехника   Розетки и выключатели   Gunsa